<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [ ]:
from esper.prelude import *
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
import pyro.optim as optim
import pyro.infer as infer
from torch.utils.data import DataLoader
from transcript_utils import *
from timeit import default_timer as now
from custom_mlp import MLP, Exp
import itertools

In [ ]:
mi_dict = {ngram: score for [ngram, score] in mutual_info('immigration')}
mi_priors = torch.tensor([mi_dict[ngram] if ngram in mi_dict else 0 for ngram in vocabulary])

In [ ]:
for k in ['immigration', 'border', 'healthcare']:
    print('{} {:.4f}'.format(k, mi_priors[vocabulary.index(k)].item()))

In [ ]:
# compute_vectors(video_list(), vocabulary, SEGMENT_SIZE, SEGMENT_STRIDE)

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self, p):
        # p = number of features
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p, 1)
        self.sigmoid = nn.Sigmoid()
        self.softplus = nn.Softplus()
        self.p = p

    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
    def model(self, x, y):
        # Create unit normal priors over the parameters
        loc, scale = torch.zeros(self.p), torch.ones(self.p) * 10
        bias_loc, bias_scale = torch.zeros(1), torch.ones(1) * 10
        w_prior = dist.Normal(loc, scale).independent(1)
        b_prior = dist.Normal(bias_loc, bias_scale).independent(1)
        priors = {'linear.weight': w_prior, 'linear.bias': b_prior}
        # lift module parameters to random variables sampled from the priors
        lifted_module = pyro.random_module("module", self, priors)
        # sample a regressor (which also samples w and b)
        lifted_reg_model = lifted_module()
        with pyro.iarange("map", x.shape[0]):
            # run the regressor forward conditioned on data
            prediction_mean = lifted_reg_model(x).squeeze(-1)
            # condition on the observed data
            pyro.sample("obs", dist.Bernoulli(prediction_mean), obs=y)                                                

    def guide(self, x, y):
        # define our variational parameters
        w_loc = torch.tensor(mi_priors)
        # note that we initialize our scales to be pretty narrow
        w_log_sig = torch.tensor(-3.0 * torch.ones(1, self.p) + 0.05 * torch.randn(1, self.p))
        b_loc = torch.tensor(0.5) + 0.05 * torch.randn(1)
        b_log_sig = torch.tensor(-3.0 * torch.ones(1) + 0.05 * torch.randn(1))
        # register learnable params in the param store
        mw_param = pyro.param("guide_mean_weight", w_loc)
        sw_param = self.softplus(pyro.param("guide_log_scale_weight", w_log_sig))
        mb_param = pyro.param("guide_mean_bias", b_loc)
        sb_param = self.softplus(pyro.param("guide_log_scale_bias", b_log_sig))
        # guide distributions for w and b
        w_dist = dist.Normal(mw_param, sw_param).independent(1)
        b_dist = dist.Normal(mb_param, sb_param).independent(1)
        dists = {'linear.weight': w_dist, 'linear.bias': b_dist}
        # overload the parameters in the module with random samples
        # from the guide distributions
        lifted_module = pyro.random_module("module", self, dists)
        # sample a regressor (which also samples w and b)
        return lifted_module()  

In [ ]:
unsup_dataset = SegmentVectorDataset(video_list(), vocab_size=vocab_size, inmemory=True)
text_dataset = SegmentTextDataset(video_list())
unsup_loader = DataLoader(unsup_dataset, batch_size=100, shuffle=True)

In [ ]:
def get_accuracy(model_gen, x, y, iters=100):
    samples = []
    for _ in range(iters):
        model = model_gen()
        y_pred = model(x).squeeze(-1).round()
        fp = torch.sum((y_pred != y) & (y_pred == 1)).item()
        fn =  torch.sum((y_pred != y) & (y_pred == 0)).item()
        acc = torch.sum(y_pred == y).item()
        n = float(y_pred.shape[0])
        samples.append(torch.tensor([acc/n, fp/n, fn/n]))
    return torch.mean(torch.stack(samples), dim=0).tolist(), torch.std(torch.stack(samples), dim=0).tolist(), 

In [ ]:
def format_data(labels):
    dataset = LabeledSegmentDataset(unsup_dataset, labels, categories=2)

    x_data, y_data, _ = unzip(list(dataset))
    y_data = torch.tensor([y[1] for y in y_data])
    x_data = torch.stack(x_data)
    
    split = int(len(x_data) * 2 / 3)

    (train_x, val_x) = (x_data[:split], x_data[split:])
    (train_y, val_y) = (y_data[:split], y_data[split:])
    
    return train_x, train_y, val_x, val_y

def add_labels(labels, train_x, train_y, val_x, val_y):
    active_train_x, active_train_y, active_val_x, active_val_y = format_data(labels)
    train_x = torch.cat((train_x, active_train_x))
    train_y = torch.cat((train_y, active_train_y))
    val_x = torch.cat((val_x, active_val_x))
    val_y = torch.cat((val_y, active_val_y))
    return train_x, train_y, val_x, val_y

In [ ]:
# mi_priors_raw = torch.tensor([mi_dict[ngram] if ngram in mi_dict else 0 for ngram in vocabulary])
# def baseline_model(x):
#     return torch.mm(x, mi_priors_raw.unsqueeze(0).t()).squeeze()
# acc = get_accuracy(lambda: baseline_model,
#              torch.cat((train_x, val_x)), torch.cat((train_y, val_y)), iters=2)[0][0]

# print('Baseline accuracy: {:.4f}'.format(acc))

In [ ]:
model_name = 'regression_active'
regression_model = RegressionModel(vocab_size)

def weights_path(iteration, epoch):
    return '/app/data/models/transcript_{}_weights_iter{:02d}_epoch{:04d}.pt'.format(model_name, iteration, epoch)

def best_weights_path(iteration):
    return '/app/data/models/transcript_{}_best_weights_iter{:02d}.pt'.format(model_name, iteration)

def torch_trainer():
    loss_fn = nn.MSELoss(size_average=False)
    optim = torch.optim.Adam(regression_model.parameters(), lr=0.05)
    
    def step(x, y):
        # run the model forward on the data
        y_pred = regression_model(x).squeeze(-1)
        # calculate the mse loss
        loss = loss_fn(y_pred, y)
        # initialize gradients to zero
        optim.zero_grad()
        # backpropagate
        loss.backward()
        # take a gradient step
        optim.step()
        return loss.item()
        
    return step, lambda: regression_model

def pyro_trainer():
    pyro.clear_param_store()
    opt = optim.Adam({"lr": 0.01})
    svi = infer.SVI(regression_model.model, regression_model.guide, opt, loss=infer.Trace_ELBO())
    
    def step(x, y):
        return svi.step(x, y)
    
    return step, lambda: regression_model.guide(None, None)
        
def train(iteration, step, model_gen, train_x, train_y, val_x, val_y, epochs=100, checkpoint_frequency=5, verbose=False):
    accs = []
    for epoch in range(epochs):
        loss = step(train_x, train_y)
        if epoch % checkpoint_frequency == 0:
            [tacc, tfp, tfn], _ = get_accuracy(model_gen, train_x, train_y)
            [vacc, vfp, vfn], [vaccstd, vfpstd, vfnstd] = get_accuracy(model_gen, val_x, val_y)
            if verbose:
                print("[iteration %04d] loss: %.0f, train: acc %.3f, val: acc %.3f (+/- %.3f) fp %.3f (+/- %.3f) fn %.3f (+/ %.3f)" % 
                      (epoch, loss, tacc, vacc, vaccstd, vfp, vfpstd, vfn, vfnstd))
            pyro.get_param_store().save(weights_path(iteration, epoch))
            accs.append(vacc)
    best_epoch = torch.tensor(accs).argmax().item() * checkpoint_frequency
    pyro.get_param_store().load(weights_path(iteration, best_epoch))
    pyro.get_param_store().save(best_weights_path(iteration))
    return best_epoch

def model_uncertainty(model_gen, x, iters=20):    
    ys_pred = []
    for _ in range(iters):
        model = model_gen()
        ys_pred.append(model(x).squeeze(-1).round())
    return torch.stack(ys_pred).std(dim=0)

def max_uncertainty(model_gen, batches=None):
    all_std = []
    all_idx = []
    loader = itertools.islice(unsup_loader, batches) if batches is not None else unsup_loader
    for x, i in tqdm(loader):
        all_std.append(model_uncertainty(model_gen, x))
        all_idx.append(i)

    all_std = torch.cat(all_std)
    all_idx = torch.cat(all_idx)
    
    top_std, top_idx = all_std.topk(1000)
    top_idx = top_idx.tolist()
    random.shuffle(top_idx)
    
    return top_idx

In [ ]:
step, model_gen = pyro_trainer()
print(train(0, step, model_gen, *format_data(pcache.get('labeled_segments')), verbose=True))

In [ ]:
train_x, train_y, val_x, val_y = format_data(pcache.get('labeled_segments'))

In [ ]:
step, model_gen = pyro_trainer()

def train_and_label(iteration, train_x, train_y, val_x, val_y):
    best_epoch = train(iteration, step, model_gen, train_x, train_y, val_x, val_y)
    
    def eval_model(it):        
        pyro.get_param_store().load(best_weights_path(it))
        [vacc, vfp, vfn], [vaccstd, vfpstd, vfnstd] = get_accuracy(model_gen, val_x, val_y, iters=1000)
        print("[iteration %d] acc %.3f (+/- %.3f) fp %.3f (+/- %.3f) fn %.3f (+/ %.3f)" % 
            (it, vacc, vaccstd, vfp, vfpstd, vfn, vfnstd))
        
    if iteration > 0:
        eval_model(iteration - 1)
    eval_model(iteration)
    
    indices = max_uncertainty(model_gen)
    
    def done_callback(labels):
        print('Added {} labels'.format(len(labels)))
        pcache.set('active_labels_{}'.format(iteration), labels)
        train_and_label(iteration + 1, *add_labels(labels, train_x, train_y, val_x, val_y))
    
    label_widget(text_dataset, indices, done_callback)
    
train_and_label(0, *format_data(pcache.get('labeled_segments')))

In [ ]:
def eval_model(path):
    pyro.get_param_store().load(path)
    old_model = RegressionModel(vocab_size)
    return get_accuracy((lambda: old_model.guide(None, None)), val_x, val_y, iters=1000)

In [ ]:
eval_model('/app/data/models/transcript_regression_weights_epoch00080.pt')

In [ ]:
eval_model('/app/data/models/transcript_regression_active_weights_epoch00060.pt')

In [ ]:
data = regression_model.guide(None, None)(x_data).squeeze(-1)
print(data)
print(data.round())

In [ ]:
list(regression_model.named_parameters())

In [ ]:
# Inspect learned parameters
print("Learned parameters:")
for name, param in regression_model.named_parameters():
    if name == 'linear.weight':
        weights = param.data.numpy().squeeze()
        idx = np.argsort(weights)[::-1]
        print(weights[idx])
        print(np.array(vocabulary)[idx][:100])